In [13]:
import PyPDF2
import pandas as pd
from langchain_groq import ChatGroq
llm=ChatGroq(model="mixtral-8x7b-32768",
              temperature=0,
              groq_api_key="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")

In [14]:
def extract_text_from_pdf(pdf_path):
    try:
        with open(pdf_path,"rb") as file:
            reader=PyPDF2.PdfReader(file)
            text=""
            for page in reader.pages:
                text+=page.extract_text()
        return text
    except Exception as e:
        print(f"Error Reading PDF : {e}")
        return ""

In [15]:
# Function to query the content
def query_pdf_content(pdf_content, query):
    prompt = f"""
    You are a PDF content assistant. The following text is extracted from a PDF document:
    ---
    {pdf_content}
    ---
    Answer the user's query based on the above content. If the answer is not found in the text, reply with 'No information exists.'
    
    Query: {query}
    """
    response = llm.invoke(prompt, max_tokens=150)
    return response.content.strip()  # Accessing the content attribute and applying strip()

In [16]:
def read_excel_info(file_path):
    """
    Reads an Excel file and returns its details, including sheet names, column headers, and data.
    
    :param file_path: Path to the Excel file
    :return: Dictionary containing sheet-wise details
    """
    excel_data = pd.ExcelFile(file_path)  # Load the Excel file
    info = {}
    
    info['sheet_names'] = excel_data.sheet_names  # Get all sheet names
    info['sheets'] = {}
    
    for sheet in excel_data.sheet_names:
        df = pd.read_excel(excel_data, sheet_name=sheet)  # Read each sheet
        info['sheets'][sheet] = {
            'columns': df.columns.tolist(),  # Get column headers
            'data': df.to_dict(orient='records')  # Convert data to list of dictionaries
        }
    
    return info

In [ ]:
# Main Execution
pdf_path = "STUDENTS.xlsx"  # Replace with your PDF file path
#pdf_content = extract_text_from_pdf(pdf_path)
pdf_content = read_excel_info(pdf_path)

if pdf_content:
    while True:
        user_query = input("Ask your question (or type 'exit' to quit): ")
        if user_query.lower() == "exit":
            break
        response = query_pdf_content(pdf_content, user_query)
        print("Response:", response)
else:
    print("Could not extract content from the PDF.")
